#### Purpose : D. MODEL DEVELOPMENT: HANDLING OUTLIERS AND MISSING VALUES, DATA NORMALIZATION, FEATURE SELECTION, HYPERPARAMETER TUNING VIA GRID SEARCH, AND PREVENTION OF DATA LEAKAGE

In this phase, the focus is on preparing the dataset and building predictive models to accurately forecast student academic performance. Key preprocessing steps include handling outliers and missing values, normalizing data where necessary, and performing feature selection to retain the most informative features while reducing redundancy. To optimize model performance, hyperparameter tuning is conducted using GridSearchCV with 5-fold cross-validation. The models selected for training include Support Vector Machines (SVM), Logistic Regression, Random Forest, XGBoost, and Gradient Boosting, providing a mix of linear, non-linear, and ensemble approaches to capture complex patterns in the data. Care is taken throughout to prevent data leakage, ensuring that all transformations and tuning steps are applied only on training data within the cross-validation framework.